# 区域卷积神经网络 Region based CNN (R-CNN)

1. Selective Search 启发式搜索 - 早于神经网络的复杂算法
2. 每一个锚框，当做一张图片，再输入预训练好的CNN模型，抽特征
3. 训练SVM分类器做分类
4. 锚框到真实边缘框之间的偏移

<img src="./pic/R-CNN.PNG" width=400 height=400>

- Q：因为每个锚框选择的大小不同，如何使不同大小的锚框，变成一个batch
- A：RoI pooling


## 兴趣区域（RoI）池化层
- 块1：max(0, 1, 4, 5) = 5
- 块2：max(2, 6) = 6
- 块3：max(8, 9) = 9
- 块4：max(10) = 10
<img src="./pic/RoI.PNG" width=400 height=400>

- Q：假如图片有1000个锚框，是否要做1000次CNN？
- A：计算量太大，解决办法：Fast RCNN，不再对每一个锚框进行特征抽取。
- 本来是先锚框再抽取，现在是先抽取整体特征，再按锚框比例配给

## Fast R-CNN

- 蓝色框：先用CNN对整张图片（不是对锚框）抽特征。得到一些7x7或者14x14的feature map
- 图片中的红色框：再selective search，比如搜到2个锚框（猫狗）
- 红色框：把这2个锚框映射到CNN输出的feature map
- 绿色框：用RoI pooling对每个锚框，抽取（输出）特征（假如输出特征是长为4的向量）
- 黄色框：假设图片中有100个锚框，则有100x4个向量（暂时先不管通道数）
- 再进入全连接层dense layer（不需要SVM对每个锚框做分类）
- 为什么Fast RCNN比RCNN快？
    - CNN不再对每一个锚框抽取特征，而是对整个图片抽取特征
    - 原始图片中的100个锚框，是要映射到CNN输出的14x14的feature map里，锚框不通过CNN
<img src="./pic/Fast R-CNN.PNG" width=400 height=400>


## Faster R-CNN

- 使用一个区域提议网络（RPN，Region proposal network）来替代启发式搜索(selective search, heuristic)来获得更好的锚框
- 相当于先用神经网络（代替selective search），训练一个比较糙的锚框
    - RPN：随机生成锚框，训练一个二分类的问题，判断锚框是否框柱物体。如果框中物体，能否更近似准确
    - NMS：去除冗余的类似的锚框，使锚框数量变少
- RPN预测出来的锚框位置，CNN给出特征图，进入RoI pooling
- 之后与Fast RCNN相同
- 相当于2次预测：第一次糙，第二次精准

<img src="./pic/Faster R-CNN.PNG" width=400 height=400>


## Mask R-CNN

- `RoI align`替换`RoI pooling`
    - 之所以不用RoI pooling，是因为假设3x3的锚框图，pooling成2x2之后，会导致像素无法完全分割（整切），则会有偏移。像素级的偏移会导致标号预测不准
    - 假设3x3的锚框，中间对半切之后，中间像素加权分成4分，作用于每一个2x2小块
<img src="./pic/Mask R-CNN1.PNG" width=400 height=400>


## Faster R-CNN性能

- 应用场景：特别关心精度，不关心速度时
- 工业关心速度
<img src="./pic/Faster R-CNN性能.PNG" width=400 height=400>


## 总结

- R-CNN是最早，也是最有名的一类基于锚框和CNN的目标检测算法
- Fast/Faster R-CNN持续提升性能
- Faster R-CNN和Mask R-CNN是在追求高精度场景下常用算法

# 单发多框检测（Single Shot Detection，SSD）

- 对比faster R-CNN 需要做`2次`预测，SSD只用做`1次`预测
<img src="./pic/单发多框.PNG" width=400 height=400>


## 生成锚框
- 对每个像素，生成多个以它为中心的锚框
- 给定 $n$ 个$s_1, \ldots, s_n$和 $m$ 个高宽比，那么生成 $n + m - 1$ 个锚框，其大小和高宽比分别为：
    - $(s_1, r_1),(s_2, r_1),\ldots, (s_n, r_1),(s_1, r_2),\ldots, (s_1, r_m1)$
<img src="./pic/SSD生成锚框.PNG" width=400 height=400>

## SSD模型

- 直接做预测，不需要做2次stage
    - SSD通过做多个分辨率图片下的锚框，来提升预测效果
- 一个基础网络来抽取特征，然后多个卷积层块来减半高宽
- 在每段都生成锚框
    - 底部段用来拟合小物体，顶部段用来拟合大物体
    - 为什么上面可以预测大物体，是因为感受野大嘛？
- 对每个锚框预测类别和边缘框
<img src="./pic/SSD模型.PNG" width=400 height=400>
<img src="./pic/SSD性能.PNG" width=300 height=300>

## 总结
- SSD通过单神经网络来检测模型
- 以每个像素为中心的产生多个锚框
- 在多个段的输出上进行多尺度的检测
    - 下层检测小物体
    - 上层检测大物体

## YOLO（你只看一次）

- SSD中锚框大量重叠（以每个像素为中心生成锚框，2个相邻的锚框的重叠率很高），因此浪费了很多计算
- YOLO尽量让锚框不重叠：
- YOLO将图片**均匀**分成$S * S$个锚框
    - 行6块，列6块：6x6
    - 每一块是一个锚框，所以锚框不会重叠，也不会浪费计算量
- 每个锚框预测B个边缘框
    - 防止B个物体，都与锚框有关
- 后续版本（V2，V3，V4...）有持续改进

<img src="./pic/YOLO锚框1.PNG" width=300 height=300>
<img src="./pic/YOLO性能.PNG" width=300 height=300>

## center net

- 非基于锚框的算法：好处是简单很多
- 主要思路：对每个像素预测，（FCN，做语义分割时，对每个像素做标号），预测的是当前像素点是不是bounding box的中心点。把锚框换算成基于像素的编号